 Purpose: <br> 1) Creating a model with NLP using NaiveBayes model to predict the wine variety

 Question: <br> 2) Which model (SVM, NLP-NaiveBayes, Logistic Regressions, Decision trees, Random Forest) will be best predicted 
              for wine varietal given a description.


 Results of NLP-NaiveBayes: <br> 3) Accuracy of model at predicting wine variety was: 0.5831475350072413


 

# Data Engineering by reading, cleaning and formating

In [2]:
# Dependecncies
from pyspark.sql import SparkSession
import pandas as pd

In [4]:
# Creating a Spark session to build an app
spark = SparkSession.builder.appName('test').getOrCreate()

In [6]:
# Import csv of input Wine Survey data from https://www.kaggle.com/zynicide/wine-reviews
start_data = spark.read.format("csv").option("header", "true").load("./InputData/winemag-data-130k-v2.csv")
start_data.show()
type(start_data)

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|_c0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87| 15.0|            Douro|

pyspark.sql.dataframe.DataFrame

In [7]:
# Create a length column to be used as a future feature 
from pyspark.sql.functions import length # length of each charater in each review/description.
data = start_data.withColumn('length', length(start_data['description']))
# data.show(truncate = False)
data.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+------+
|_c0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|length|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|   172|
|  1| Portugal|This is ripe and ...|            Avidagos|   

In [9]:
sparktoPD = data.toPandas()
sparktoPD.head()

,_c0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,length
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,None,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,172.0
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,227.0
2,2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,186.0
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,199.0
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,249.0


In [10]:
sparktoPD.drop(sparktoPD.columns[[0]], axis=1, inplace=True) # drop that unnamed row numbers column
sparktoPD.describe(include='all')

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,length
count,129912,129974,92513,129966,120976,129909,108734,50521,103726,98762,129965,129963,129963,129974.000000
unique,47,119958,37995,43,414,449,1252,38,35,32,118831,727,16774,NaN
top,US,"Seductively tart in lemon pith, cranberry and ...",Reserve,88,20.0,California,Napa Valley,Central Coast,Roger Voss,@vossroger,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Wines & Winemakers,NaN
freq,54504,3,2009,17203,6938,36240,4477,11063,25513,25513,11,13270,222,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.561089
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.596748
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,282.000000


In [13]:
# Checking the top 20 wines which has the highest frequencies.
varieties =  sparktoPD['variety'].value_counts()
varieties

Pinot Noir                                                 13270
Chardonnay                                                 11752
Cabernet Sauvignon                                          9467
Red Blend                                                   8945
Bordeaux-style Red Blend                                    6914
Riesling                                                    5184
Sauvignon Blanc                                             4965
Syrah                                                       4142
Rosé                                                        3563
Merlot                                                      3102
Nebbiolo                                                    2803
Zinfandel                                                   2714
Sangiovese                                                  2707
Malbec                                                      2652
Portuguese Red                                              2466
White Blend              

In [14]:
# considering only the top 20 wines with the highest frequencies. As many of the wine varietals have less than 5 reviews
top_wines_df =  sparktoPD.loc[ sparktoPD['variety'].isin(varieties.axes[0][:20])] # first column[0],  rows :20  
top_wines_df['variety'].describe()

count          93895
unique            20
top       Pinot Noir
freq           13270
Name: variety, dtype: object

In [15]:
data

DataFrame[_c0: string, country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, taster_name: string, taster_twitter_handle: string, title: string, variety: string, winery: string, length: int]

In [27]:
#convert pd to spark
spark_df = spark.createDataFrame(top_wines_df)
spark_df.head(5)

[Row(country='Italy', description="Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", designation='Vulkà Bianco', points='87', price=None, province='Sicily & Sardinia', region_1='Etna', region_2=None, taster_name='Kerin O’Keefe', taster_twitter_handle='@kerinokeefe', title='Nicosia 2013 Vulkà Bianco  (Etna)', variety='White Blend', winery='Nicosia', length=172.0),
 Row(country='Portugal', description="This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.", designation='Avidagos', points='87', price='15.0', province='Douro', region_1=None, region_2=None, taster_name='Roger Voss', taster_twitter_handle='@vossroger', title='Quinta dos Avidagos 2011 Avidagos Red (Douro)', variety='Portuguese Red',

In [17]:
len(sparktoPD['variety'].unique())

728

# Feature Transformations

In [20]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

# Create all the features to the data set
wine_variety_to_num = StringIndexer(inputCol= 'variety',outputCol='label')
tokenizer = Tokenizer(inputCol="description", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

# By default the number of features can be a million 1,000,000 but my local host cannot handle computing 
# was getting out of memory issue of heap. restricting "numFeatures=10000"
# hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token', numFeatures=1000)
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token', numFeatures=10000)
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
# See https://spark.apache.org/docs/latest/ml-features.html#vectorassembler
# This just creates a new, single vector of features that is the concatenation
# of tf-idf data and the length of the wine varietal

clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [22]:
# Create and run a data processing Pipeline
# See https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline

from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[wine_variety_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [23]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(spark_df)

# Running the transform method here _actually_ runs the pipeline on our 
# DataFrame and returns the transformed data
cleaned = cleaner.transform(spark_df)

In [24]:
# Show label of wine variety and resulting features
cleaned.select(['label', 'features']).show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                    

In [28]:
# Break data down into a training set and a testing set
(training, testing) = cleaned.randomSplit([0.7, 0.3])

In [29]:
training.count()

65842

In [43]:
testing.count()

28053

# Importing the NLP NaiveBayes model

In [32]:
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel

# Create a Naive Bayes model and fit training data
nb = NaiveBayes(smoothing=1.0, modelType='multinomial')
wine_predictor = nb.fit(training)

In [33]:
# When using the pyspark.mllib.classification had lots of issues with Java errors as NaiveBayesModel belongs to 
# pyspark.ml.classification library.
from pyspark.ml.classification import NaiveBayesModel

In [34]:
# Tranform the model with the testing data
test_results = wine_predictor.transform(testing)
test_results.show(5)

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+-----------------+---------------------+--------------------+---------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  country|         description|         designation|points|price|        province|            region_1|region_2|      taster_name|taster_twitter_handle|               title|        variety|   winery|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+-----------------+---------------------+--------------------+---------------+---------+------+-----+--------------------+--------------------+-

In [35]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting wine variety was: %s" % (acc))

Accuracy of model at predicting wine variety was: 0.5831475350072413


### Accuracy of model at predicting wine variety was: 0.5831475350072413

# Saving a Trained Model
We can save our trained models using model.save("model name")

In [81]:
# Save the model
# Note: model name is NOT "nb" 
# wine.predictor is the model name as it is trained model.
wine_predictor.save("NaiveBayes_model.sav")

# Checking to see if the model is created with data and metadata

In [82]:
ls # current working directory


Database Engineering.ipynb
InputData/
Logistic_Model.sav
ML-DecisionTreeClassifier-&-RandomForestClassifier.ipynb
NLP-NaiveBayes.ipynb
NaiveBayes_model/
NaiveBayes_model.sav/
PySpark_and_NLP.pptx
README
app.py
my_model/
saved_model/
spark-warehouse/
templates/
testing/
wines.sqlite


In [85]:
!ls NaiveBayes_model.sav/

data     metadata


In [86]:
!open NaiveBayes_model.sav/

In [87]:
#!rm -r NLP_NaiveBayes_trained/

In [88]:
!ls
    

Database Engineering.ipynb
InputData
Logistic_Model.sav
ML-DecisionTreeClassifier-&-RandomForestClassifier.ipynb
NLP-NaiveBayes.ipynb
NaiveBayes_model
NaiveBayes_model.sav
PySpark_and_NLP.pptx
README
app.py
my_model
saved_model
spark-warehouse
templates
testing
wines.sqlite


# Loading the created model to test 

In [89]:
myNaiveBayes_model = NaiveBayesModel.load("NaiveBayes_model.sav")

In [90]:
test_model_results = myNaiveBayes_model.transform(testing)
test_model_results.show(5)

 

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+-----------------+---------------------+--------------------+---------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  country|         description|         designation|points|price|        province|            region_1|region_2|      taster_name|taster_twitter_handle|               title|        variety|   winery|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------+-----------------+---------------------+--------------------+---------------+---------+------+-----+--------------------+--------------------+-

# Predicting the model

In [91]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

In [92]:
le = LabelEncoder()
y = le.fit_transform(top_wines_df['variety'])

In [93]:
# term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important 
# a word is to a document in a collections
from sklearn.feature_extraction.text import TfidfVectorizer


#Here, we use the Tfidf Vectorizer to figure out the importance of the words that make up the descriptions given
from sklearn.linear_model import LogisticRegression  
vect = TfidfVectorizer()
X = vect.fit_transform(top_wines_df['description'])

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 101)

In [95]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
NBmodel = MultinomialNB()
NBmodel.fit(X_train, y_train)
NBmodel.predict(X_test)
le.classes_[NBmodel.predict(X_test)[0:20]]

array(['Pinot Noir', 'Cabernet Sauvignon', 'Red Blend',
       'Cabernet Sauvignon', 'Bordeaux-style Red Blend', 'Chardonnay',
       'Pinot Noir', 'Pinot Noir', 'Chardonnay', 'Chardonnay', 'Red Blend',
       'Chardonnay', 'Bordeaux-style Red Blend', 'Cabernet Sauvignon',
       'Red Blend', 'Chardonnay', 'Cabernet Sauvignon', 'Chardonnay',
       'Cabernet Sauvignon', 'Chardonnay'], dtype=object)